In [1]:
##################################################################################
##### Define all parameters for model tuning
##################################################################################

n_fold = 5
expName = "NT_Site_PredNTS_Classification_DLNN_Kgap"
outPath = "Results"
foldName = "folds.pickle"

# epochs = 50
# batch_size = 32
shuffle = True
seed = None

input_data_folder = "PredNTS_MathFeature_ENC"

# monitor = 'val_loss'

In [2]:
kgap_max = 4

train_data_filename = 'Training-datasets-PredNTS_kgap_{}.csv'
indpe_data_filename = 'independent-dataset-PredNTS_kgap_{}.csv'

In [3]:
import os 
import pickle
import numpy as np
import pandas as pd

import tensorflow as tf

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_curve, auc, accuracy_score, precision_score, confusion_matrix
from sklearn.metrics import roc_auc_score, classification_report, matthews_corrcoef

import math

In [4]:
# print(tf.test.is_gpu_available(cuda_only=True))
# physical_devices = tf.config.experimental.list_physical_devices('GPU')
physical_devices = tf.config.list_physical_devices('GPU')
print(physical_devices)
tf.config.experimental.set_memory_growth(physical_devices[0], True)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [5]:
# ##################################################################################
# ##### define all CUSTOM functions
# ##################################################################################

# def one_hot_encode_nt(sequence, char_dict):
    
#     seq_encoded = np.zeros((len(sequence),len(char_dict)))
    
#     i = 0
#     for single_character in sequence:
#         if(single_character.upper() in char_dict.keys()):
#             seq_encoded[i][char_dict[single_character.upper()]] = 1
#             i = i+1
#         else:
#             raise ValueError('Incorrect character in NT sequence: '+sequence)
#     return seq_encoded

In [6]:
##################################################################################
##### Build k-fold functions
##################################################################################

## Build the K-fold from dataset
def build_kfold(features, labels, k=10, shuffle=False, seed=None):
    
    skf = StratifiedKFold(n_splits=k, shuffle=shuffle, random_state=seed)
    kfoldList = []
    for train_index, test_index in skf.split(features, labels):
        X_train, X_test = features[train_index], features[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
        kfoldList.append({
            "X_train": X_train,
            "X_test": X_test,
            "y_train":y_train,
            "y_test":y_test
        })
    return kfoldList

In [7]:
##################################################################################
##### define evaluator functions
##################################################################################

def pred2label(y_pred):
    y_pred = np.round(y_pred)
    return y_pred

In [8]:
monitor = 'val_accuracy'

epochs = 100
batch_size = 32

##################################################################################
##### Function to customize the DLNN architecture with parameters
##################################################################################

def DLNN_Classifier(input_vec_shape,
                    dense_decode_units = 512, ## Dense layer parameters
                    prob = 0.5, learn_rate = 0.0005, loss = 'binary_crossentropy', metrics = 'accuracy'):
    
    beta = 0.001
    
    input1 = tf.keras.layers.Input(shape=input_vec_shape)
    
    ######################################################################################################
    ########  Classifier  ################################################################################
    ######################################################################################################
    
    y = tf.keras.layers.Dense(dense_decode_units, 
                              kernel_regularizer = tf.keras.regularizers.l2(beta))(input1)
    y = tf.keras.layers.GaussianNoise(stddev=0.1)(y)
    y = tf.keras.layers.Dropout(prob)(y)
    
    y = tf.keras.layers.Dense(int(dense_decode_units/2), 
                              kernel_regularizer = tf.keras.regularizers.l2(beta), 
                              activation = 'relu')(y)
    y = tf.keras.layers.GaussianNoise(stddev=0.1)(y)
    y = tf.keras.layers.Dropout(prob)(y)
    
    y = tf.keras.layers.Dense(int(dense_decode_units/4), 
                              kernel_regularizer = tf.keras.regularizers.l2(beta), 
                              activation = 'relu')(y)
    y = tf.keras.layers.GaussianNoise(stddev=0.1)(y)
    y = tf.keras.layers.Dropout(prob)(y)
    
    y = tf.keras.layers.Dense(1, 
                              kernel_regularizer = tf.keras.regularizers.l2(beta), 
                              activation = 'sigmoid')(y)

    ## Generate Model from input and output
    model = tf.keras.models.Model(inputs=input1, outputs=y)
    
    ## Compile model
    if(metrics != None):
        model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=learn_rate), 
                      loss = loss, metrics = metrics)
    else:
        model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=learn_rate), 
                      loss = loss)

    return model

In [9]:
# monitor = 'val_loss'

# epochs = 100
# batch_size = 32

# ##################################################################################
# ##### Function to customize the DLNN architecture with parameters
# ##################################################################################

# def DLNN_Classifier(input_vec_shape,
#                     dense_decode_units = 512, ## Dense layer parameters,
#                     dense_layers = 5,
#                     prob = 0.5, learn_rate = 0.0005, 
#                     gn = 0.1,
#                     loss = 'binary_crossentropy', metrics = 'accuracy'):
    
#     beta = 0.001
    
#     input1 = tf.keras.layers.Input(shape=input_vec_shape)
    
#     ######################################################################################################
#     ########  Classifier  ################################################################################
#     ######################################################################################################
    
#     y = tf.keras.layers.Dense(dense_decode_units, 
#                               kernel_regularizer = tf.keras.regularizers.l2(beta))(input1)
#     y = tf.keras.layers.GaussianNoise(stddev=gn)(y)
#     y = tf.keras.layers.Dropout(prob)(y)
    
#     for i in range(1,dense_layers+1):
    
#         y = tf.keras.layers.Dense(int(dense_decode_units/(2**i)), 
#                                   kernel_regularizer = tf.keras.regularizers.l2(beta), 
#                                  )(y)
#         y = tf.keras.layers.GaussianNoise(stddev=gn)(y)
#         y = tf.keras.layers.Dropout(prob)(y) 
    
#     y = tf.keras.layers.Dense(1, 
#                               kernel_regularizer = tf.keras.regularizers.l2(beta), 
#                               activation = 'sigmoid')(y)

#     ## Generate Model from input and output
#     model = tf.keras.models.Model(inputs=input1, outputs=y)
    
#     ## Compile model
#     if(metrics != None):
#         model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=learn_rate), 
#                       loss = loss, metrics = metrics)
#     else:
#         model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=learn_rate), 
#                       loss = loss)

#     return model

In [10]:
# monitor = 'val_loss'

# epochs = 100
# batch_size = 32

# ##################################################################################
# ##### Function to customize the DLNN architecture with parameters
# ##################################################################################

# def DLNN_Classifier(input_vec_shape,
#                     dense_decode_units = 128, ## Dense layer parameters
#                     prob = 0.5, learn_rate = 0.0001, loss = 'binary_crossentropy', metrics = 'accuracy'):
    
#     beta = 0.001
    
#     input1 = tf.keras.layers.Input(shape=input_vec_shape)
    
#     ######################################################################################################
#     ########  Classifier  ################################################################################
#     ######################################################################################################
    
#     y = tf.keras.layers.Dense(dense_decode_units, 
#                               kernel_regularizer = tf.keras.regularizers.l2(beta))(input1)
    
#     y = tf.keras.layers.Dropout(prob)(y)
    
#     y = tf.keras.layers.Dense(int(dense_decode_units/2), 
#                               kernel_regularizer = tf.keras.regularizers.l2(beta), 
#                               activation = 'relu')(y)
    
#     y = tf.keras.layers.Dropout(prob)(y)
    
#     y = tf.keras.layers.Dense(int(dense_decode_units/4), 
#                               kernel_regularizer = tf.keras.regularizers.l2(beta), 
#                               activation = 'relu')(y)
    
#     y = tf.keras.layers.Dropout(prob)(y)
    
#     y = tf.keras.layers.Dense(1, 
#                               kernel_regularizer = tf.keras.regularizers.l2(beta), 
#                               activation = 'sigmoid')(y)

#     ## Generate Model from input and output
#     model = tf.keras.models.Model(inputs=input1, outputs=y)
    
#     ## Compile model
#     if(metrics != None):
#         model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=learn_rate), 
#                       loss = loss, metrics = metrics)
#     else:
#         model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=learn_rate), 
#                       loss = loss)

#     return model

In [11]:
# epochs = 200
# batch_size = 16

# ##################################################################################
# ##### Function to customize the DLNN architecture with parameters
# ##################################################################################

# def DLNN_Classifier(input_vec_shape,
#                     dense_decode_units = 8, ## Dense layer parameters,
#                     dense_layers = 2,
#                     prob = 0.5, learn_rate = 0.0001, loss = 'binary_crossentropy', metrics = 'accuracy'):
    
#     beta = 0.001
    
#     input1 = tf.keras.layers.Input(shape=input_vec_shape)
    
#     ######################################################################################################
#     ########  Classifier  ################################################################################
#     ######################################################################################################
    
#     y = tf.keras.layers.Dense(dense_decode_units, 
#                               kernel_regularizer = tf.keras.regularizers.l2(beta))(input1)
#     y = tf.keras.layers.BatchNormalization()(y)
#     y = tf.keras.layers.GaussianNoise(stddev=0.1)(y)
#     y = tf.keras.layers.Dropout(prob)(y)
    
#     for i in range(1,dense_layers+1):
    
#         y = tf.keras.layers.Dense(int(dense_decode_units/(2**i)), 
#                                   kernel_regularizer = tf.keras.regularizers.l2(beta), 
#                                  )(y)
#         y = tf.keras.layers.BatchNormalization()(y)
#         y = tf.keras.layers.GaussianNoise(stddev=0.1)(y)
#         y = tf.keras.layers.Dropout(prob)(y)
    
#     y = tf.keras.layers.Dense(1, 
#                               kernel_regularizer = tf.keras.regularizers.l2(beta), 
#                               activation = 'sigmoid')(y)

#     ## Generate Model from input and output
#     model = tf.keras.models.Model(inputs=input1, outputs=y)
    
#     ## Compile model
#     if(metrics != None):
#         model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=learn_rate), 
#                       loss = loss, metrics = metrics)
#     else:
#         model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=learn_rate), 
#                       loss = loss)

#     return model

In [12]:
# for step in range(10):
#     initial_learning_rate=1e-1
#     decay_steps=10000
#     decay_rate=0.9
#     print(step, ':', initial_learning_rate * decay_rate ** (step / decay_steps))

In [13]:
DLNN_Classifier((2000)).summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 2000)]            0         
                                                                 
 dense (Dense)               (None, 512)               1024512   
                                                                 
 gaussian_noise (GaussianNoi  (None, 512)              0         
 se)                                                             
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 gaussian_noise_1 (GaussianN  (None, 256)              0         
 oise)                                                       

# Train data preparation

In [14]:
##################################################################################
##### Read CSV data
##################################################################################

for i in range(kgap_max+1):
    
    current_train_data_filepath = os.path.join(input_data_folder, train_data_filename.format(i))
    current_train_data = pd.read_csv(current_train_data_filepath, sep=',', header=0)
    current_train_data = current_train_data.drop('label', axis=1)
    
    if i == 0:
        train_data = current_train_data
    else:
        train_data = pd.merge(
            train_data,
            current_train_data,
            how="inner",
            on='nameseq'
        )

train_data['label'] = pd.Series([int(val.split('_')[-2])
                                 for val in train_data['nameseq']])

train_data = train_data.drop('nameseq', axis=1)

##################################################################################
##### Extract features and labels, create folds
##################################################################################

train_features = np.array(train_data.drop('label', axis=1))
train_labels = np.array(train_data['label'])
train_labels = train_labels.reshape((train_labels.shape[0], 1))

input_vec_shape = train_features[0].shape

folds = build_kfold(train_features, train_labels, k=n_fold, shuffle=shuffle, seed=seed)

## Write the k-fold dataset to file
foldPath = os.path.join(outPath, expName, "{}fold".format(n_fold))
if(not os.path.isdir(foldPath)):
    os.makedirs(foldPath)
pickle.dump(folds, open(os.path.join(foldPath, foldName), "wb"))

# Training

In [15]:
##################################################################################
##### For each input file, train model and generate different outputs in a structured folder
##################################################################################

## create the evaluation data structure for all iterations
evaluations = {
    "Fold" : [],
    "Train_Test" : [],
    "Accuracy" : [],
    "Precision": [],
    "TPR": [],
    "FPR": [],
    "TPR_FPR_Thresholds": [],
    "AUC": [],
    "Sensitivity": [],
    "Specificity": [],
    "MCC":[]
}

##################################################################################
##### Train/Test model on all folds, generate evaluations
##################################################################################

## Create and set directory to save model
modelPath = os.path.join(outPath, expName, "{}fold".format(n_fold), "models")
if(not os.path.isdir(modelPath)):
    os.makedirs(modelPath)

i = -1
for fold in folds:
    i += 1
    
    print("\nTrain/Test model on Fold #"+str(i)+".")
    
    model = DLNN_Classifier(input_vec_shape = input_vec_shape)
    
    ## Define the model callbacks for early stopping and saving the model. Then train model
    current_model_path = os.path.join(modelPath, "bestModel-fold{}.hdf5".format(i))
    modelCallbacks = [
        tf.keras.callbacks.ModelCheckpoint(current_model_path,
                                           monitor = monitor, verbose = 1, save_best_only = True, 
                                           save_weights_only = False, mode = 'auto', save_freq = 'epoch'),
    ]
    
    # adding random shuffling of the dataset for training purpose
    index_arr = np.arange(fold["X_train"].shape[0])
    index_arr = np.random.permutation(index_arr)
    
    model.fit(x = fold["X_train"][index_arr], y = fold["y_train"][index_arr], 
              batch_size = batch_size, epochs = epochs, verbose = 1, 
              callbacks = modelCallbacks, validation_data = (fold["X_test"], fold["y_test"]))
    
    model = tf.keras.models.load_model(current_model_path)
    
    ##################################################################################
    ##### Prediction and metrics for TRAIN dataset
    ##################################################################################

    y_pred = model.predict(fold["X_train"])
    label_pred = pred2label(y_pred)
    
    # Compute precision, recall, sensitivity, specifity, mcc
    acc = accuracy_score(fold["y_train"], label_pred)
    prec = precision_score(fold["y_train"],label_pred)
    mcc = matthews_corrcoef(fold["y_train"], label_pred)

    conf = confusion_matrix(fold["y_train"], label_pred)
    tn, fp, fn, tp = conf.ravel()
    sens = tp/(tp+fn)
    spec = tn/(tn+fp)
    
    fpr, tpr, thresholds = roc_curve(fold["y_train"], y_pred)
    auc = roc_auc_score(fold["y_train"], y_pred)
    
    evaluations["Fold"].append(i)
    evaluations["Train_Test"].append("Train")
    evaluations["Accuracy"].append(acc)
    evaluations["Precision"].append(prec)
    evaluations["TPR"].append(tpr)
    evaluations["FPR"].append(fpr)
    evaluations["TPR_FPR_Thresholds"].append(thresholds)
    evaluations["AUC"].append(auc)
    evaluations["Sensitivity"].append(sens)
    evaluations["Specificity"].append(spec)
    evaluations["MCC"].append(mcc)
    
    ##################################################################################
    ##### Prediction and metrics for TEST dataset
    ##################################################################################

    y_pred = model.predict(fold["X_test"])
    label_pred = pred2label(y_pred)
    
    # Compute precision, recall, sensitivity, specifity, mcc
    acc = accuracy_score(fold["y_test"], label_pred)
    prec = precision_score(fold["y_test"],label_pred)
    mcc = matthews_corrcoef(fold["y_test"], label_pred)

    conf = confusion_matrix(fold["y_test"], label_pred)
    tn, fp, fn, tp = conf.ravel()
    sens = tp/(tp+fn)
    spec = tn/(tn+fp)
    
    fpr, tpr, thresholds = roc_curve(fold["y_test"], y_pred)
    auc = roc_auc_score(fold["y_test"], y_pred)
    
    evaluations["Fold"].append(i)
    evaluations["Train_Test"].append("Test")
    evaluations["Accuracy"].append(acc)
    evaluations["Precision"].append(prec)
    evaluations["TPR"].append(tpr)
    evaluations["FPR"].append(fpr)
    evaluations["TPR_FPR_Thresholds"].append(thresholds)
    evaluations["AUC"].append(auc)
    evaluations["Sensitivity"].append(sens)
    evaluations["Specificity"].append(spec)
    evaluations["MCC"].append(mcc)
    
    del model
    tf.keras.backend.clear_session()


Train/Test model on Fold #0.
Epoch 1/100
57/60 [===========================>..] - ETA: 0s - loss: 2.0259 - accuracy: 0.5757
Epoch 1: val_accuracy improved from -inf to 0.77358, saving model to Results\NT_Site_PredNTS_Classification_DLNN_Kgap\5fold\models\bestModel-fold0.hdf5
60/60 [==============================] - 2s 9ms/step - loss: 2.0227 - accuracy: 0.5774 - val_loss: 1.8182 - val_accuracy: 0.7736
Epoch 2/100
58/60 [============================>.] - ETA: 0s - loss: 1.8078 - accuracy: 0.7247
Epoch 2: val_accuracy did not improve from 0.77358
60/60 [==============================] - 0s 5ms/step - loss: 1.8115 - accuracy: 0.7228 - val_loss: 1.7152 - val_accuracy: 0.7694
Epoch 3/100
60/60 [==============================] - ETA: 0s - loss: 1.6633 - accuracy: 0.7953
Epoch 3: val_accuracy improved from 0.77358 to 0.83648, saving model to Results\NT_Site_PredNTS_Classification_DLNN_Kgap\5fold\models\bestModel-fold0.hdf5
60/60 [==============================] - 0s 6ms/step - loss: 1.6633 -

49/60 [=======================>......] - ETA: 0s - loss: 0.5182 - accuracy: 0.9770
Epoch 29: val_accuracy did not improve from 0.85115
60/60 [==============================] - 0s 5ms/step - loss: 0.5155 - accuracy: 0.9774 - val_loss: 1.5331 - val_accuracy: 0.7987
Epoch 30/100
58/60 [============================>.] - ETA: 0s - loss: 0.5181 - accuracy: 0.9725
Epoch 30: val_accuracy did not improve from 0.85115
60/60 [==============================] - 0s 5ms/step - loss: 0.5175 - accuracy: 0.9727 - val_loss: 1.3671 - val_accuracy: 0.8092
Epoch 31/100
59/60 [============================>.] - ETA: 0s - loss: 0.4849 - accuracy: 0.9804
Epoch 31: val_accuracy did not improve from 0.85115
60/60 [==============================] - 0s 5ms/step - loss: 0.4847 - accuracy: 0.9806 - val_loss: 1.4728 - val_accuracy: 0.7987
Epoch 32/100
60/60 [==============================] - ETA: 0s - loss: 0.4592 - accuracy: 0.9816
Epoch 32: val_accuracy did not improve from 0.85115
60/60 [===========================

Epoch 59/100
48/60 [=======================>......] - ETA: 0s - loss: 0.2608 - accuracy: 0.9805
Epoch 59: val_accuracy did not improve from 0.85115
60/60 [==============================] - 0s 5ms/step - loss: 0.2643 - accuracy: 0.9795 - val_loss: 1.3664 - val_accuracy: 0.7925
Epoch 60/100
49/60 [=======================>......] - ETA: 0s - loss: 0.2676 - accuracy: 0.9796
Epoch 60: val_accuracy did not improve from 0.85115
60/60 [==============================] - 0s 5ms/step - loss: 0.2639 - accuracy: 0.9811 - val_loss: 1.3982 - val_accuracy: 0.7862
Epoch 61/100
58/60 [============================>.] - ETA: 0s - loss: 0.2612 - accuracy: 0.9795
Epoch 61: val_accuracy did not improve from 0.85115
60/60 [==============================] - 0s 5ms/step - loss: 0.2622 - accuracy: 0.9790 - val_loss: 1.1736 - val_accuracy: 0.7862
Epoch 62/100
52/60 [=========================>....] - ETA: 0s - loss: 0.2396 - accuracy: 0.9832
Epoch 62: val_accuracy did not improve from 0.85115
60/60 [==============

Epoch 89/100
48/60 [=======================>......] - ETA: 0s - loss: 0.2167 - accuracy: 0.9805
Epoch 89: val_accuracy did not improve from 0.85115
60/60 [==============================] - 0s 5ms/step - loss: 0.2195 - accuracy: 0.9801 - val_loss: 1.3429 - val_accuracy: 0.7966
Epoch 90/100
49/60 [=======================>......] - ETA: 0s - loss: 0.2154 - accuracy: 0.9841
Epoch 90: val_accuracy did not improve from 0.85115
60/60 [==============================] - 0s 5ms/step - loss: 0.2151 - accuracy: 0.9843 - val_loss: 1.3443 - val_accuracy: 0.7966
Epoch 91/100
49/60 [=======================>......] - ETA: 0s - loss: 0.2176 - accuracy: 0.9821
Epoch 91: val_accuracy did not improve from 0.85115
60/60 [==============================] - 0s 5ms/step - loss: 0.2204 - accuracy: 0.9795 - val_loss: 1.1357 - val_accuracy: 0.8050
Epoch 92/100
49/60 [=======================>......] - ETA: 0s - loss: 0.2113 - accuracy: 0.9860
Epoch 92: val_accuracy did not improve from 0.85115
60/60 [==============

60/60 [==============================] - ETA: 0s - loss: 0.7945 - accuracy: 0.9622
Epoch 16: val_accuracy did not improve from 0.81971
60/60 [==============================] - 0s 5ms/step - loss: 0.7945 - accuracy: 0.9622 - val_loss: 1.7609 - val_accuracy: 0.7883
Epoch 17/100
60/60 [==============================] - ETA: 0s - loss: 0.7703 - accuracy: 0.9669
Epoch 17: val_accuracy did not improve from 0.81971
60/60 [==============================] - 0s 5ms/step - loss: 0.7703 - accuracy: 0.9669 - val_loss: 1.6526 - val_accuracy: 0.8155
Epoch 18/100
52/60 [=========================>....] - ETA: 0s - loss: 0.7332 - accuracy: 0.9706
Epoch 18: val_accuracy did not improve from 0.81971
60/60 [==============================] - 0s 5ms/step - loss: 0.7296 - accuracy: 0.9717 - val_loss: 1.7306 - val_accuracy: 0.8071
Epoch 19/100
59/60 [============================>.] - ETA: 0s - loss: 0.6947 - accuracy: 0.9783
Epoch 19: val_accuracy did not improve from 0.81971
60/60 [===========================

...

Epoch 82/100
53/60 [=========================>....] - ETA: 0s - loss: 0.1833 - accuracy: 0.9876
Epoch 82: val_accuracy did not improve from 0.77731
60/60 [==============================] - 0s 5ms/step - loss: 0.1833 - accuracy: 0.9874 - val_loss: 1.8650 - val_accuracy: 0.7626
Epoch 83/100
52/60 [=========================>....] - ETA: 0s - loss: 0.1948 - accuracy: 0.9814
Epoch 83: val_accuracy did not improve from 0.77731
60/60 [==============================] - 0s 5ms/step - loss: 0.1978 - accuracy: 0.9811 - val_loss: 1.7878 - val_accuracy: 0.7584
Epoch 84/100
51/60 [========================>.....] - ETA: 0s - loss: 0.1897 - accuracy: 0.9859
Epoch 84: val_accuracy did not improve from 0.77731
60/60 [==============================] - 0s 5ms/step - loss: 0.1893 - accuracy: 0.9853 - val_loss: 1.9380 - val_accuracy: 0.7542
Epoch 85/100
51/60 [========================>.....] - ETA: 0s - loss: 0.1892 - accuracy: 0.9865
Epoch 85: val_accuracy did not improve from 0.77731
60/60 [==============

60/60 [==============================] - 0s 5ms/step - loss: 1.0614 - accuracy: 0.9496 - val_loss: 1.6305 - val_accuracy: 0.7668
Epoch 11/100
58/60 [============================>.] - ETA: 0s - loss: 0.9952 - accuracy: 0.9601
Epoch 11: val_accuracy did not improve from 0.77941
60/60 [==============================] - 0s 5ms/step - loss: 0.9950 - accuracy: 0.9596 - val_loss: 1.7121 - val_accuracy: 0.7563
Epoch 12/100
59/60 [============================>.] - ETA: 0s - loss: 0.9542 - accuracy: 0.9629
Epoch 12: val_accuracy did not improve from 0.77941
60/60 [==============================] - 0s 5ms/step - loss: 0.9544 - accuracy: 0.9627 - val_loss: 1.7264 - val_accuracy: 0.7626
Epoch 13/100
59/60 [============================>.] - ETA: 0s - loss: 0.9049 - accuracy: 0.9666
Epoch 13: val_accuracy did not improve from 0.77941
60/60 [==============================] - 0s 5ms/step - loss: 0.9041 - accuracy: 0.9669 - val_loss: 1.8029 - val_accuracy: 0.7647
Epoch 14/100
60/60 [====================

57/60 [===========================>..] - ETA: 0s - loss: 0.3514 - accuracy: 0.9830
Epoch 40: val_accuracy did not improve from 0.77941
60/60 [==============================] - 0s 6ms/step - loss: 0.3543 - accuracy: 0.9816 - val_loss: 1.8230 - val_accuracy: 0.7332
Epoch 41/100
49/60 [=======================>......] - ETA: 0s - loss: 0.3446 - accuracy: 0.9802
Epoch 41: val_accuracy did not improve from 0.77941
60/60 [==============================] - 0s 5ms/step - loss: 0.3426 - accuracy: 0.9811 - val_loss: 1.9524 - val_accuracy: 0.7269
Epoch 42/100
59/60 [============================>.] - ETA: 0s - loss: 0.3437 - accuracy: 0.9799
Epoch 42: val_accuracy did not improve from 0.77941
60/60 [==============================] - 0s 5ms/step - loss: 0.3439 - accuracy: 0.9795 - val_loss: 1.5992 - val_accuracy: 0.7395
Epoch 43/100
60/60 [==============================] - ETA: 0s - loss: 0.3511 - accuracy: 0.9774
Epoch 43: val_accuracy did not improve from 0.77941
60/60 [===========================

Epoch 70/100
50/60 [========================>.....] - ETA: 0s - loss: 0.2196 - accuracy: 0.9869
Epoch 70: val_accuracy did not improve from 0.77941
60/60 [==============================] - 0s 5ms/step - loss: 0.2183 - accuracy: 0.9869 - val_loss: 1.8638 - val_accuracy: 0.7458
Epoch 71/100
59/60 [============================>.] - ETA: 0s - loss: 0.2112 - accuracy: 0.9878
Epoch 71: val_accuracy did not improve from 0.77941
60/60 [==============================] - 0s 5ms/step - loss: 0.2111 - accuracy: 0.9879 - val_loss: 1.8981 - val_accuracy: 0.7479
Epoch 72/100
49/60 [=======================>......] - ETA: 0s - loss: 0.2108 - accuracy: 0.9841
Epoch 72: val_accuracy did not improve from 0.77941
60/60 [==============================] - 0s 5ms/step - loss: 0.2158 - accuracy: 0.9811 - val_loss: 1.7082 - val_accuracy: 0.7374
Epoch 73/100
57/60 [===========================>..] - ETA: 0s - loss: 0.2184 - accuracy: 0.9819
Epoch 73: val_accuracy did not improve from 0.77941
60/60 [==============

Epoch 100/100
59/60 [============================>.] - ETA: 0s - loss: 0.1826 - accuracy: 0.9868
Epoch 100: val_accuracy did not improve from 0.77941
60/60 [==============================] - 0s 5ms/step - loss: 0.1853 - accuracy: 0.9864 - val_loss: 1.7242 - val_accuracy: 0.7206


## k-fold Training evaluation

In [16]:
evaluations_df = pd.DataFrame.from_dict(evaluations)

evaluations_df_grouped = evaluations_df.groupby(["Train_Test"]).mean().filter(['Accuracy', 
                                                                               'Precision', 
                                                                               'AUC', 
                                                                               'Sensitivity', 
                                                                               'Specificity', 
                                                                               'MCC'])

evaluations_df_grouped

,Accuracy,Precision,AUC,Sensitivity,Specificity,MCC
Train_Test,,,,,,
Test,0.80392,0.812516,0.87682,0.790900,0.816947,0.608185
Train,0.95760,0.958669,0.99184,0.956965,0.958235,0.915524


In [17]:

# Accuracy	Precision	AUC	Sensitivity	Specificity	MCC
# Train_Test						
# Test	0.795964	0.811851	0.855731	0.770782	0.821153	0.592854
# Train	0.974601	0.979426	0.995659	0.969561	0.979639	0.949266

In [18]:
# 	Accuracy	Precision	AUC	Sensitivity	Specificity	MCC
# Train_Test						
# Test	0.796817	0.809338	0.875483	0.777490	0.816104	0.594429
# Train	0.931568	0.941050	0.981401	0.920863	0.942273	0.863358

In [19]:
evaluations_df[evaluations_df["Train_Test"] == 'Test']

,Fold,Train_Test,Accuracy,Precision,TPR,FPR,TPR_FPR_Thresholds,AUC,Sensitivity,Specificity,MCC
1,0,Test,0.851153,0.868421,"[0.0, 0.0041841004184100415, 0.171548117154811...","[0.0, 0.0, 0.0, 0.004201680672268907, 0.004201...","[1.9997654, 0.99976546, 0.98726636, 0.98704356...",0.918445,0.828452,0.873950,0.703082
3,1,Test,0.819706,0.827586,"[0.0, 0.004201680672268907, 0.0210084033613445...","[0.0, 0.0, 0.0, 0.0041841004184100415, 0.00418...","[2.0, 1.0, 0.9999988, 0.99999857, 0.9999981, 0...",0.880173,0.806723,0.832636,0.639595
5,2,Test,0.792017,0.806167,"[0.0, 0.004201680672268907, 0.0840336134453781...","[0.0, 0.0, 0.0, 0.004201680672268907, 0.004201...","[1.9983857, 0.99838567, 0.9903127, 0.9892468, ...",0.874073,0.768908,0.815126,0.584658
7,3,Test,0.777311,0.775000,"[0.0, 0.004201680672268907, 0.0420168067226890...","[0.0, 0.0, 0.0, 0.004201680672268907, 0.004201...","[1.999928, 0.999928, 0.99861467, 0.99853396, 0...",0.852465,0.781513,0.773109,0.554641
9,4,Test,0.779412,0.785408,"[0.0, 0.004201680672268907, 0.0546218487394958...","[0.0, 0.0, 0.0, 0.004201680672268907, 0.004201...","[2.0, 1.0, 0.9999554, 0.99995434, 0.99796677, ...",0.858944,0.768908,0.789916,0.558947


In [20]:
# Fold	Train_Test	Accuracy	Precision	TPR	FPR	TPR_FPR_Thresholds	AUC	Sensitivity	Specificity	MCC
# 1	0	Test	0.798742	0.817778	[0.0, 0.0041841004184100415, 0.016736401673640...	[0.0, 0.0, 0.0, 0.004201680672268907, 0.004201...	[1.974611, 0.97461104, 0.95518667, 0.94677013,...	0.853548	0.769874	0.827731	0.598564
# 3	1	Test	0.807128	0.820175	[0.0, 0.004201680672268907, 0.2605042016806723...	[0.0, 0.0, 0.0, 0.0041841004184100415, 0.00418...	[1.983566, 0.9835661, 0.8779062, 0.87508714, 0...	0.868728	0.785714	0.828452	0.614761
# 5	2	Test	0.796218	0.816143	[0.0, 0.004201680672268907, 0.1470588235294117...	[0.0, 0.0, 0.0, 0.004201680672268907, 0.004201...	[1.9717295, 0.97172946, 0.87369037, 0.8617482,...	0.852800	0.764706	0.827731	0.593617
# 7	3	Test	0.787815	0.812785	[0.0, 0.004201680672268907, 0.0378151260504201...	[0.0, 0.0, 0.0, 0.004201680672268907, 0.004201...	[1.9766836, 0.97668356, 0.951985, 0.9506801, 0...	0.854115	0.747899	0.827731	0.577473
# 9	4	Test	0.789916	0.792373	[0.0, 0.004201680672268907, 0.0378151260504201...	[0.0, 0.0, 0.0, 0.004201680672268907, 0.004201...	[1.9974229, 0.99742293, 0.9586073, 0.95359516,...	0.849463	0.785714	0.794118	0.579852

In [21]:
evaluations_df

,Fold,Train_Test,Accuracy,Precision,TPR,FPR,TPR_FPR_Thresholds,AUC,Sensitivity,Specificity,MCC
0,0,Train,0.945932,0.953042,"[0.0, 0.0010504201680672268, 0.036764705882352...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.9999784, 0.9999784, 0.9992091, 0.99919873, ...",0.990261,0.938025,0.953830,0.891973
1,0,Test,0.851153,0.868421,"[0.0, 0.0041841004184100415, 0.171548117154811...","[0.0, 0.0, 0.0, 0.004201680672268907, 0.004201...","[1.9997654, 0.99976546, 0.98726636, 0.98704356...",0.918445,0.828452,0.873950,0.703082
2,1,Train,0.984777,0.987342,"[0.0, 0.005246589716684155, 0.0094438614900314...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.9999999, 0.9999999, 0.99999964, 0.99999917,...",0.998908,0.982162,0.987395,0.969567
3,1,Test,0.819706,0.827586,"[0.0, 0.004201680672268907, 0.0210084033613445...","[0.0, 0.0, 0.0, 0.0041841004184100415, 0.00418...","[2.0, 1.0, 0.9999988, 0.99999857, 0.9999981, 0...",0.880173,0.806723,0.832636,0.639595
4,2,Train,0.938090,0.949408,"[0.0, 0.001049317943336831, 0.0786988457502623...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.9994674, 0.9994673, 0.9893952, 0.9891722, 0...",0.985172,0.925498,0.950682,0.876458
5,2,Test,0.792017,0.806167,"[0.0, 0.004201680672268907, 0.0840336134453781...","[0.0, 0.0, 0.0, 0.004201680672268907, 0.004201...","[1.9983857, 0.99838567, 0.9903127, 0.9892468, ...",0.874073,0.768908,0.815126,0.584658
6,3,Train,0.935467,0.914172,"[0.0, 0.001049317943336831, 0.0870933892969569...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.9999124, 0.9999124, 0.99653757, 0.99652195,...",0.985887,0.961175,0.909759,0.872087
7,3,Test,0.777311,0.775000,"[0.0, 0.004201680672268907, 0.0420168067226890...","[0.0, 0.0, 0.0, 0.004201680672268907, 0.004201...","[1.999928, 0.999928, 0.99861467, 0.99853396, 0...",0.852465,0.781513,0.773109,0.554641
8,4,Train,0.983736,0.989384,"[0.0, 0.002098635886673662, 0.0083945435466946...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[2.0, 1.0, 0.99999976, 0.9999995, 0.99999917, ...",0.998971,0.977964,0.989507,0.967536
9,4,Test,0.779412,0.785408,"[0.0, 0.004201680672268907, 0.0546218487394958...","[0.0, 0.0, 0.0, 0.004201680672268907, 0.004201...","[2.0, 1.0, 0.9999554, 0.99995434, 0.99796677, ...",0.858944,0.768908,0.789916,0.558947


# Independent data

In [22]:
##################################################################################
##### Read CSV data
##################################################################################

for i in range(kgap_max+1):

    current_indpe_data_filepath = os.path.join(input_data_folder, indpe_data_filename.format(i))
    current_indpe_data = pd.read_csv(current_indpe_data_filepath, sep=',', header=0)
    current_indpe_data = current_indpe_data.drop('label', axis=1)
    
    if i == 0:
        indpe_data = current_indpe_data
    else:
        indpe_data = pd.merge(
            indpe_data,
            current_indpe_data,
            how="inner",
            on='nameseq'
        )

indpe_data['label'] = pd.Series([int(val.split('_')[-2])
                                 for val in indpe_data['nameseq']])

indpe_data = indpe_data.drop('nameseq', axis=1)

##################################################################################
##### Extract features and labels, create folds
##################################################################################

indpe_features = np.array(indpe_data.drop('label', axis=1))
indpe_labels = np.array(indpe_data['label'])
indpe_labels = indpe_labels.reshape((indpe_labels.shape[0], 1))

## Using k-fold Models

### Performance of each k-fold model

In [23]:
## create the evaluation data structure for all iterations
evaluations = {
    "Fold" : [],
    "Train_Test" : [],
    "Accuracy" : [],
    "Precision": [],
    "TPR": [],
    "FPR": [],
    "TPR_FPR_Thresholds": [],
    "AUC": [],
    "Sensitivity": [],
    "Specificity": [],
    "MCC":[]
}

##################################################################################
##### Prediction and metrics for Independent dataset
##################################################################################

for i in range(n_fold):
    
    current_model_path = os.path.join(modelPath, "bestModel-fold{}.hdf5".format(i))
    model = tf.keras.models.load_model(current_model_path)

    y_pred = model.predict(indpe_features)
    label_pred = pred2label(y_pred)

    # Compute precision, recall, sensitivity, specifity, mcc
    acc = accuracy_score(indpe_labels, label_pred)
    prec = precision_score(indpe_labels,label_pred)
    mcc = matthews_corrcoef(indpe_labels, label_pred)

    conf = confusion_matrix(indpe_labels, label_pred)
    tn, fp, fn, tp = conf.ravel()
    sens = tp/(tp+fn)
    spec = tn/(tn+fp)
    
    fpr, tpr, thresholds = roc_curve(indpe_labels, y_pred)
    auc = roc_auc_score(indpe_labels, y_pred)

    evaluations["Fold"].append(i)
    evaluations["Train_Test"].append("Independent")
    evaluations["Accuracy"].append(acc)
    evaluations["Precision"].append(prec)
    evaluations["TPR"].append(tpr)
    evaluations["FPR"].append(fpr)
    evaluations["TPR_FPR_Thresholds"].append(thresholds)
    evaluations["AUC"].append(auc)
    evaluations["Sensitivity"].append(sens)
    evaluations["Specificity"].append(spec)
    evaluations["MCC"].append(mcc)
    
    del model
    tf.keras.backend.clear_session()

##################################################################################

evaluations_df = pd.DataFrame.from_dict(evaluations)

evaluations_df_grouped = evaluations_df.groupby(["Train_Test"]).mean().filter(['Accuracy', 
                                                                               'Precision', 
                                                                               'AUC', 
                                                                               'Sensitivity', 
                                                                               'Specificity', 
                                                                               'MCC'])

evaluations_df_grouped

,Accuracy,Precision,AUC,Sensitivity,Specificity,MCC
Train_Test,,,,,,
Independent,0.622857,0.244492,0.66552,0.608867,0.625636,0.177439


In [24]:
evaluations_df

,Fold,Train_Test,Accuracy,Precision,TPR,FPR,TPR_FPR_Thresholds,AUC,Sensitivity,Specificity,MCC
0,0,Independent,0.619592,0.237525,"[0.0, 0.0049261083743842365, 0.004926108374384...","[0.0, 0.0, 0.0009784735812133072, 0.0009784735...","[1.9998792, 0.99987924, 0.9998493, 0.99967635,...",0.658898,0.586207,0.626223,0.160658
1,1,Independent,0.622857,0.243564,"[0.0, 0.0049261083743842365, 0.009852216748768...","[0.0, 0.0, 0.0, 0.0019569471624266144, 0.00195...","[2.0, 1.0, 0.99999964, 0.99999857, 0.99999475,...",0.669018,0.605911,0.626223,0.175348
2,2,Independent,0.647347,0.259958,"[0.0, 0.0049261083743842365, 0.014778325123152...","[0.0, 0.0, 0.0, 0.005870841487279843, 0.005870...","[1.9990268, 0.9990268, 0.9979796, 0.9941326, 0...",0.673166,0.610837,0.654599,0.202406
3,3,Independent,0.586939,0.238342,"[0.0, 0.0049261083743842365, 0.009852216748768...","[0.0, 0.0, 0.0, 0.0009784735812133072, 0.00097...","[1.9999044, 0.9999044, 0.9998957, 0.9998043, 0...",0.666326,0.679803,0.568493,0.184922
4,4,Independent,0.637551,0.243070,"[0.0, 0.0049261083743842365, 0.009852216748768...","[0.0, 0.0009784735812133072, 0.001956947162426...","[1.9999998, 0.99999976, 0.9999995, 0.99999833,...",0.660190,0.561576,0.652642,0.163864


### Mean score with k-fold models

In [25]:
## create the evaluation data structure for all iterations
evaluations = {
    "Train_Test" : [],
    "Accuracy" : [],
    "Precision": [],
    "TPR": [],
    "FPR": [],
    "TPR_FPR_Thresholds": [],
    "AUC": [],
    "Sensitivity": [],
    "Specificity": [],
    "MCC":[]
}

##################################################################################
##### Prediction and metrics for Independent dataset
##################################################################################

total_pred = np.zeros(indpe_labels.shape)
all_preds = []

for i in range(n_fold):
    
    current_model_path = os.path.join(modelPath, "bestModel-fold{}.hdf5".format(i))
    model = tf.keras.models.load_model(current_model_path)

    y_pred = model.predict(indpe_features)
    total_pred += y_pred
    all_preds.append(y_pred)
    
    del model
    tf.keras.backend.clear_session()
    
total_pred = total_pred / n_fold
label_pred = pred2label(total_pred)

# Compute precision, recall, sensitivity, specifity, mcc
acc = accuracy_score(indpe_labels, label_pred)
prec = precision_score(indpe_labels,label_pred)
mcc = matthews_corrcoef(indpe_labels, label_pred)

conf = confusion_matrix(indpe_labels, label_pred)
tn, fp, fn, tp = conf.ravel()
sens = tp/(tp+fn)
spec = tn/(tn+fp)

fpr, tpr, thresholds = roc_curve(indpe_labels, total_pred)
auc = roc_auc_score(indpe_labels, total_pred)

evaluations["Train_Test"].append("Independent")
evaluations["Accuracy"].append(acc)
evaluations["Precision"].append(prec)
evaluations["TPR"].append(tpr)
evaluations["FPR"].append(fpr)
evaluations["TPR_FPR_Thresholds"].append(thresholds)
evaluations["AUC"].append(auc)
evaluations["Sensitivity"].append(sens)
evaluations["Specificity"].append(spec)
evaluations["MCC"].append(mcc)

##################################################################################

evaluations_df = pd.DataFrame.from_dict(evaluations)

evaluations_df_grouped = evaluations_df.groupby(["Train_Test"]).mean().filter(['Accuracy', 
                                                                               'Precision', 
                                                                               'AUC', 
                                                                               'Sensitivity', 
                                                                               'Specificity', 
                                                                               'MCC'])

evaluations_df_grouped

,Accuracy,Precision,AUC,Sensitivity,Specificity,MCC
Train_Test,,,,,,
Independent,0.631837,0.245902,0.673816,0.591133,0.639922,0.175487


### Voting score with k-fold models

In [26]:
## create the evaluation data structure for all iterations
evaluations = {
    "Train_Test" : [],
    "Accuracy" : [],
    "Precision": [],
    "TPR": [],
    "FPR": [],
    "TPR_FPR_Thresholds": [],
    "AUC": [],
    "Sensitivity": [],
    "Specificity": [],
    "MCC":[]
}

##################################################################################
##### Prediction and metrics for Independent dataset
##################################################################################

total_pred = np.zeros(indpe_labels.shape)
all_preds = []

for i in range(n_fold):
    
    current_model_path = os.path.join(modelPath, "bestModel-fold{}.hdf5".format(i))
    model = tf.keras.models.load_model(current_model_path)

    y_pred = model.predict(indpe_features)
    vote_pred = pred2label(y_pred)
    total_pred += vote_pred
    all_preds.append(vote_pred)
    
    del model
    tf.keras.backend.clear_session()
    
total_pred = total_pred / n_fold
label_pred = pred2label(total_pred)

# Compute precision, recall, sensitivity, specifity, mcc
acc = accuracy_score(indpe_labels, label_pred)
prec = precision_score(indpe_labels,label_pred)
mcc = matthews_corrcoef(indpe_labels, label_pred)

conf = confusion_matrix(indpe_labels, label_pred)
tn, fp, fn, tp = conf.ravel()
sens = tp/(tp+fn)
spec = tn/(tn+fp)

fpr, tpr, thresholds = roc_curve(indpe_labels, total_pred)
auc = roc_auc_score(indpe_labels, total_pred)

evaluations["Train_Test"].append("Independent")
evaluations["Accuracy"].append(acc)
evaluations["Precision"].append(prec)
evaluations["TPR"].append(tpr)
evaluations["FPR"].append(fpr)
evaluations["TPR_FPR_Thresholds"].append(thresholds)
evaluations["AUC"].append(auc)
evaluations["Sensitivity"].append(sens)
evaluations["Specificity"].append(spec)
evaluations["MCC"].append(mcc)

##################################################################################

evaluations_df = pd.DataFrame.from_dict(evaluations)

evaluations_df_grouped = evaluations_df.groupby(["Train_Test"]).mean().filter(['Accuracy', 
                                                                               'Precision', 
                                                                               'AUC', 
                                                                               'Sensitivity', 
                                                                               'Specificity', 
                                                                               'MCC'])

evaluations_df_grouped

,Accuracy,Precision,AUC,Sensitivity,Specificity,MCC
Train_Test,,,,,,
Independent,0.630204,0.245935,0.654707,0.596059,0.636986,0.176758


## Using New Model

Train model on full data from training. Predict and evaluate on Independent data.

In [27]:
model = DLNN_Classifier(input_vec_shape = input_vec_shape)
    
## Define the model callbacks for early stopping and saving the model. Then train model
current_model_path = os.path.join(modelPath, "_fullModel.hdf5")
modelCallbacks = [
    tf.keras.callbacks.ModelCheckpoint(current_model_path,
                                       monitor = monitor, verbose = 1, save_best_only = True, 
                                       save_weights_only = False, mode = 'auto', save_freq = 'epoch'),
]

# adding random shuffling of the dataset for training purpose
index_arr = np.arange(train_features.shape[0])
index_arr = np.random.permutation(index_arr)

model.fit(x = train_features[index_arr], y = train_labels[index_arr], batch_size = batch_size, epochs = epochs, verbose = 1, 
          callbacks = modelCallbacks, validation_data = (indpe_features, indpe_labels))
# model.fit(x = train_features[index_arr], y = train_labels[index_arr], batch_size = batch_size, epochs = epochs, verbose = 1, 
#           callbacks = modelCallbacks, validation_split = 0.2)

model = tf.keras.models.load_model(current_model_path)

Epoch 1/100
64/75 [========================>.....] - ETA: 0s - loss: 2.0111 - accuracy: 0.5830
Epoch 1: val_accuracy improved from -inf to 0.61143, saving model to Results\NT_Site_PredNTS_Classification_DLNN_Kgap\5fold\models\_fullModel.hdf5
75/75 [==============================] - 1s 9ms/step - loss: 1.9938 - accuracy: 0.5915 - val_loss: 1.9509 - val_accuracy: 0.6114
Epoch 2/100
75/75 [==============================] - ETA: 0s - loss: 1.7646 - accuracy: 0.7385
Epoch 2: val_accuracy did not improve from 0.61143
75/75 [==============================] - 0s 6ms/step - loss: 1.7646 - accuracy: 0.7385 - val_loss: 1.9491 - val_accuracy: 0.6016
Epoch 3/100
71/75 [===========================>..] - ETA: 0s - loss: 1.6089 - accuracy: 0.8103
Epoch 3: val_accuracy did not improve from 0.61143
75/75 [==============================] - 0s 6ms/step - loss: 1.6145 - accuracy: 0.8081 - val_loss: 2.2376 - val_accuracy: 0.5102
Epoch 4/100
75/75 [==============================] - ETA: 0s - loss: 1.4787 - a

Epoch 30/100
69/75 [==========================>...] - ETA: 0s - loss: 0.4270 - accuracy: 0.9733
Epoch 30: val_accuracy did not improve from 0.68490
75/75 [==============================] - 0s 6ms/step - loss: 0.4286 - accuracy: 0.9731 - val_loss: 2.8155 - val_accuracy: 0.6457
Epoch 31/100
69/75 [==========================>...] - ETA: 0s - loss: 0.4130 - accuracy: 0.9746
Epoch 31: val_accuracy did not improve from 0.68490
75/75 [==============================] - 0s 6ms/step - loss: 0.4124 - accuracy: 0.9752 - val_loss: 3.0977 - val_accuracy: 0.6302
Epoch 32/100
75/75 [==============================] - ETA: 0s - loss: 0.4041 - accuracy: 0.9731
Epoch 32: val_accuracy did not improve from 0.68490
75/75 [==============================] - 0s 6ms/step - loss: 0.4041 - accuracy: 0.9731 - val_loss: 2.7611 - val_accuracy: 0.6343
Epoch 33/100
73/75 [============================>.] - ETA: 0s - loss: 0.4023 - accuracy: 0.9722
Epoch 33: val_accuracy did not improve from 0.68490
75/75 [==============

Epoch 60/100
75/75 [==============================] - ETA: 0s - loss: 0.2358 - accuracy: 0.9849
Epoch 60: val_accuracy did not improve from 0.68490
75/75 [==============================] - 0s 6ms/step - loss: 0.2358 - accuracy: 0.9849 - val_loss: 2.7457 - val_accuracy: 0.6694
Epoch 61/100
71/75 [===========================>..] - ETA: 0s - loss: 0.2563 - accuracy: 0.9758
Epoch 61: val_accuracy did not improve from 0.68490
75/75 [==============================] - 0s 6ms/step - loss: 0.2575 - accuracy: 0.9752 - val_loss: 2.7094 - val_accuracy: 0.6220
Epoch 62/100
73/75 [============================>.] - ETA: 0s - loss: 0.2633 - accuracy: 0.9756
Epoch 62: val_accuracy did not improve from 0.68490
75/75 [==============================] - 0s 6ms/step - loss: 0.2631 - accuracy: 0.9757 - val_loss: 2.2312 - val_accuracy: 0.6506
Epoch 63/100
69/75 [==========================>...] - ETA: 0s - loss: 0.2514 - accuracy: 0.9751
Epoch 63: val_accuracy did not improve from 0.68490
75/75 [==============

Epoch 90/100
71/75 [===========================>..] - ETA: 0s - loss: 0.2117 - accuracy: 0.9837
Epoch 90: val_accuracy did not improve from 0.68490
75/75 [==============================] - 0s 6ms/step - loss: 0.2130 - accuracy: 0.9832 - val_loss: 2.4098 - val_accuracy: 0.6359
Epoch 91/100
73/75 [============================>.] - ETA: 0s - loss: 0.2021 - accuracy: 0.9872
Epoch 91: val_accuracy did not improve from 0.68490
75/75 [==============================] - 0s 6ms/step - loss: 0.2028 - accuracy: 0.9870 - val_loss: 2.9453 - val_accuracy: 0.6294
Epoch 92/100
74/75 [============================>.] - ETA: 0s - loss: 0.2065 - accuracy: 0.9806
Epoch 92: val_accuracy did not improve from 0.68490
75/75 [==============================] - 0s 6ms/step - loss: 0.2063 - accuracy: 0.9807 - val_loss: 2.6993 - val_accuracy: 0.6318
Epoch 93/100
63/75 [========================>.....] - ETA: 0s - loss: 0.2107 - accuracy: 0.9861
Epoch 93: val_accuracy did not improve from 0.68490
75/75 [==============

In [28]:
## create the evaluation data structure for all iterations
evaluations = {
    "Train_Test" : [],
    "Accuracy" : [],
    "Precision": [],
    "TPR": [],
    "FPR": [],
    "TPR_FPR_Thresholds": [],
    "AUC": [],
    "Sensitivity": [],
    "Specificity": [],
    "MCC":[]
}

##################################################################################
##### Prediction and metrics for Independent dataset
##################################################################################

y_pred = model.predict(indpe_features)
label_pred = pred2label(y_pred)

# Compute precision, recall, sensitivity, specifity, mcc
acc = accuracy_score(indpe_labels, label_pred)
prec = precision_score(indpe_labels,label_pred)
mcc = matthews_corrcoef(indpe_labels, label_pred)

conf = confusion_matrix(indpe_labels, label_pred)
tn, fp, fn, tp = conf.ravel()
sens = tp/(tp+fn)
spec = tn/(tn+fp)

fpr, tpr, thresholds = roc_curve(indpe_labels, y_pred)
auc = roc_auc_score(indpe_labels, y_pred)

evaluations["Train_Test"].append("Independent")
evaluations["Accuracy"].append(acc)
evaluations["Precision"].append(prec)
evaluations["TPR"].append(tpr)
evaluations["FPR"].append(fpr)
evaluations["TPR_FPR_Thresholds"].append(thresholds)
evaluations["AUC"].append(auc)
evaluations["Sensitivity"].append(sens)
evaluations["Specificity"].append(spec)
evaluations["MCC"].append(mcc)

##################################################################################

evaluations_df = pd.DataFrame.from_dict(evaluations)

evaluations_df_grouped = evaluations_df.groupby(["Train_Test"]).mean().filter(['Accuracy', 
                                                                               'Precision', 
                                                                               'AUC', 
                                                                               'Sensitivity', 
                                                                               'Specificity', 
                                                                               'MCC'])

evaluations_df_grouped

,Accuracy,Precision,AUC,Sensitivity,Specificity,MCC
Train_Test,,,,,,
Independent,0.684898,0.268354,0.663711,0.522167,0.717221,0.190432


In [29]:
print(classification_report(indpe_labels, np.round(y_pred).astype(int)))

              precision    recall  f1-score   support

           0       0.88      0.72      0.79      1022
           1       0.27      0.52      0.35       203

    accuracy                           0.68      1225
   macro avg       0.58      0.62      0.57      1225
weighted avg       0.78      0.68      0.72      1225



In [30]:
# larger n/w
# precision    recall  f1-score   support

#            0       0.89      0.71      0.79      1022
#            1       0.27      0.55      0.37       203

#     accuracy                           0.68      1225
#    macro avg       0.58      0.63      0.58      1225
# weighted avg       0.79      0.68      0.72      1225